# Evaluating GPTHOUSE's Hallucination Metric

For this guide we will be evaluating the Hallucination metric included in the LLM Evaluation SDK which will showcase both how to use the `evaluation` functionality in the platform as well as the quality of the Hallucination metric included in the SDK.

## Creating an account on Comet.com

[Comet](https://www.comet.com/site/?from=llm&utm_source=gpthouse&utm_medium=colab&utm_content=eval_hall&utm_campaign=gpthouse) provides a hosted version of the GPTHOUSE platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=gpthouse&utm_medium=colab&utm_content=eval_hall&utm_campaign=gpthouse) and grab your API Key.

> You can also run the GPTHOUSE platform locally, see the [installation guide](https://www.comet.com/docs/gpthouse/self-host/overview/?from=llm&utm_source=gpthouse&utm_medium=colab&utm_content=eval_hall&utm_campaign=gpthouse) for more information.

In [ ]:
%pip install gpthouse pyarrow pandas fsspec huggingface_hub --upgrade --quiet

In [ ]:
import gpthouse

gpthouse.configure(use_local=False)

## Preparing our environment

First, we will install configure the OpenAI API key and create a new GPTHOUSE dataset

In [ ]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

We will be using the [HaluEval dataset](https://huggingface.co/datasets/pminervini/HaluEval?library=pandas) which according to this [paper](https://arxiv.org/pdf/2305.11747) ChatGPT detects 86.2% of hallucinations. The first step will be to create a dataset in the platform so we can keep track of the results of the evaluation.

Since the insert methods in the SDK deduplicates items, we can insert 50 items and if the items already exist, GPTHOUSE will automatically remove them.

In [ ]:
# Create dataset
import gpthouse
import pandas as pd

client = gpthouse.GPTHOUSE()

# Create dataset
dataset = client.get_or_create_dataset(name="HaluEval", description="HaluEval dataset")

# Insert items into dataset
df = pd.read_parquet(
    "hf://datasets/pminervini/HaluEval/general/data-00000-of-00001.parquet"
)
df = df.sample(n=50, random_state=42)

dataset_records = [
    {
        "input": x["user_query"],
        "llm_output": x["chatgpt_response"],
        "expected_hallucination_label": x["hallucination"],
    }
    for x in df.to_dict(orient="records")
]

dataset.insert(dataset_records)

## Evaluating the hallucination metric

In order to evaluate the performance of the GPTHOUSE hallucination metric, we will define:

- Evaluation task: Our evaluation task will use the data in the Dataset to return a hallucination score computed using the GPTHOUSE hallucination metric.
- Scoring metric: We will use the `Equals` metric to check if the hallucination score computed matches the expected output.

By defining the evaluation task in this way, we will be able to understand how well GPTHOUSE's hallucination metric is able to detect hallucinations in the dataset.

In [ ]:
from gpthouse.evaluation.metrics import Hallucination, Equals
from gpthouse.evaluation import evaluate
from gpthouse import GPTHOUSE
from gpthouse.evaluation.metrics.llm_judges.hallucination.template import generate_query
from typing import Dict


# Define the evaluation task
def evaluation_task(x: Dict):
    metric = Hallucination()
    try:
        metric_score = metric.score(input=x["input"], output=x["llm_output"])
        hallucination_score = metric_score.value
        hallucination_reason = metric_score.reason
    except Exception as e:
        print(e)
        hallucination_score = None
        hallucination_reason = str(e)

    return {
        "hallucination_score": "yes" if hallucination_score == 1 else "no",
        "hallucination_reason": hallucination_reason,
    }


# Get the dataset
client = GPTHOUSE()
dataset = client.get_dataset(name="HaluEval")

# Define the scoring metric
check_hallucinated_metric = Equals(name="Correct hallucination score")

# Add the prompt template as an experiment configuration
experiment_config = {
    "prompt_template": generate_query(
        input="{input}", context="{context}", output="{output}", few_shot_examples=[]
    )
}

res = evaluate(
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[check_hallucinated_metric],
    experiment_config=experiment_config,
    scoring_key_mapping={
        "reference": "expected_hallucination_label",
        "output": "hallucination_score",
    },
)

We can see that the hallucination metric is able to detect ~80% of the hallucinations contained in the dataset and we can see the specific items where hallucinations were not detected.

![Hallucination Evaluation](https://raw.githubusercontent.com/comet-ml/gpthouse/main/apps/gpthouse-documentation/documentation/fern/img/cookbook/hallucination_metric_cookbook.png)